In [7]:
import numpy as np
import math
from enum import Enum

from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.models import PanTool, WheelZoomTool, ResetTool, SaveTool, HoverTool

output_notebook()

k_b = 1.380649e-23
m_H = 1.67262192369e-27
m_NA = 3.81735881448E-26
m_e = 9.1093837015E-31
q_e = 1.602176634E-19

class p_type(Enum):
    Electron = 1
    Hydrogen = 2
    Natrium = 3

def runge_kutta4(v, B, h, c):
    k1 = c * np.cross(v, B)
    k2 = c * np.cross(v + 0.5 * h * k1, B)
    k3 = c * np.cross(v + 0.5 * h * k2, B)
    k4 = c * np.cross(v + h * k3, B)
    return h * (k1 / 6.0 + k2 / 3.0 + k3 / 3.0 + k4 / 6.0)

def run_sim(particle_type, z_0, T, time_period):
    if particle_type is p_type.Electron:
        q = -1.0 * q_e
        m = m_e
    elif particle_type is p_type.Hydrogen:
        q = q_e
        m = m_H
        t_step = 0.1
        #time_period = 200.0
    elif particle_type is p_type.Natrium:
        q = 2.0 * q_e
        m = m_NA
        t_step = 0.1
        #time_period = 2000.0
    N = int(time_period / t_step)
    q_per_m = q / m
    B_0 = 60e-9
    d = 0.195 * 2440000
    
    v_0 = math.sqrt(3.0 * k_b * T / m)
    
    pos = np.array([0.0, 0.0, 1000.0 * z_0])
    vel = np.array([0.0, v_0, 0.0])

    y = np.zeros([N], dtype=float)
    z = np.zeros([N], dtype=float)
    B_values = np.zeros([N], dtype=float)
    for i in range(N):
        B = np.array([B_0 * math.tanh(pos[2] / d), 0.0, 0.0])
        dv = runge_kutta4(vel, B, t_step, q_per_m)
        vel = vel + dv
        pos = pos + t_step * vel
        y[i] = pos[1]/1000.0
        z[i] = pos[2]/1000.0
        B_values[i] = B[0] * 1e9
    return [y,z,B_values]

def update(particle_type = "Natrium", z_0=100.0, T=6360.0, time_period=1000.0):
    if   particle_type == "Electron": pflag = p_type.Electron
    elif particle_type == "Hydrogen": pflag = p_type.Hydrogen
    elif particle_type == "Natrium": pflag = p_type.Natrium
    y, z, B_values = run_sim(pflag, z_0, T, time_period)
    r.data_source.data = {'x': y, 'y': z}
    p.y_range.start = -(z_0+10)
    p.y_range.end = (z_0+10)
    B_r.data_source.data = {'x': np.arange(len(B_values)), 'y': B_values}
    active_indices=np.arange(int(zrange/2+int(min(z))),int(zrange/2+int(max(z))))
    B_tan_active_r.data_source.data = {'x': zs[active_indices], 'y': Bs[active_indices]}
    push_notebook()


pan_tool = PanTool()
pan_tool.dimensions = "width"
wheel_tool = WheelZoomTool()
reset_tool = ResetTool()
save_tool = SaveTool()
hover_tool = HoverTool()

y, z, B_values = run_sim(p_type.Natrium, 100.0, 6360.0, 1000.0)
p = figure(title="Particle trajectory", plot_height=300, plot_width=900, x_range=[-1600,0], y_range=[-110,110], background_fill_color='#efefef', tools = [pan_tool, wheel_tool, reset_tool, save_tool, hover_tool])
r = p.line(y, z, color="#8888cc", line_width=3.0, alpha=0.8)
p.xaxis.axis_label = "Y position (km)"
p.yaxis.axis_label = "Z position (km)"
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1

p.toolbar.active_drag = pan_tool
p.toolbar.active_scroll = wheel_tool
p.toolbar.active_inspect = hover_tool

B_p = figure(title="Magnetic field at particle position", plot_height=300, plot_width=900, x_range=[0,len(B_values)], background_fill_color='#efefef', tools = [pan_tool, wheel_tool, reset_tool, save_tool, hover_tool])
B_r = B_p.line(np.arange(len(B_values)), B_values, color="#8888cc", line_width=3.0, alpha=0.8)

B_p.xaxis.axis_label = "Time (s)"
B_p.yaxis.axis_label = "Magnetic field strength (nT)"
B_p.ygrid.band_fill_color = "olive"
B_p.ygrid.band_fill_alpha = 0.1
B_p.add_tools(pan_tool)
B_p.toolbar.active_drag = pan_tool
B_p.toolbar.active_inspect = hover_tool

zrange=3000
d = 0.195 * 2440000
B_0 = 60
zs = np.arange(zrange) - 0.5*zrange
Bs = B_0 * np.tanh(zs * (1000.0 / d))
B_tan_p = figure(title="Magnetic field in X direction", plot_height=400, plot_width=900, x_range=[-0.5*zrange,0.5*zrange], background_fill_color='#efefef', tools = [save_tool, hover_tool])
B_tan_r = B_tan_p.line(zs, Bs, color="#8888cc", line_width=3.0, alpha=0.8)
active_indices=np.arange(int(zrange/2+int(min(z))),int(zrange/2+int(max(z))))
B_tan_active_r = B_tan_p.line(zs[active_indices], Bs[active_indices], color="#FF0000", line_width=3.0, alpha=0.8, legend_label="B-field traveled by particle")
B_tan_p.xaxis.axis_label = "Z (km)"
B_tan_p.yaxis.axis_label = "Magnetic field strength (nT)"
B_tan_p.toolbar.active_inspect = hover_tool
B_tan_p.legend.location = 'top_left'
B_tan_p.legend.title_text_font = 'Arial'
B_tan_p.legend.title_text_font_size = '20pt'

interact(update, particle_type=["Hydrogen", "Natrium"], z_0=(-0.0,200.0,1.0), T=(0.0,50000.0,500.0), time_period=(0.0,2000.0,10.0))
plot_handle = show(column(p, B_p, B_tan_p), notebook_handle=True)


Loading BokehJS ...

interactive(children=(Dropdown(description='particle_type', index=1, options=('Hydrogen', 'Natrium'), value='N…